In [1]:
#data
import pandas as pd
import os
import statsmodels.stats.multitest as multi
import numpy as np
import json

In [2]:
#plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
#statistics
import lifelines
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
import statsmodels.stats.multitest as multi
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from lifelines.statistics import logrank_test
from scipy.stats import mannwhitneyu
import statsmodels.formula.api as smf
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
import sklearn.metrics as metrics
import xgboost
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [4]:
with open('/cellar/users/mpagadal/resources/tcga/ensembl_map.json', 'r') as f:
    ensembl = json.load(f)
ensembl={k.split(".")[0]:v for k,v in ensembl.items()}

In [5]:
random_state=75

In [6]:
def make_surv_df(surv,surv_type,raw):
    
    '''
    make TCGA survival dataframe
    '''
    
    #get survival dataframe
    surv=pd.read_csv(surv,index_col=0)
    surv=surv.rename(columns={"bcr_patient_barcode":"FID"})
    surv=surv[surv["Redaction"].isnull()]
    
    #get phenos
    raw=pd.read_csv(raw,delim_whitespace=True,index_col=0)
    raw=raw.iloc[:,5:]
    raw=raw.reset_index()
    raw=raw.rename(columns={"index":"FID"})

    #combine dataframes
    full_surv=pd.merge(surv[["FID",surv_type,surv_type+".time","age_at_initial_pathologic_diagnosis","race","gender","ajcc_pathologic_tumor_stage"]], raw, on="FID", how="left")
    #full_surv=full_surv[full_surv[surv_type+".time"]<1825] #censor at 5 years
    #get patients with Stage designations
    full_surv=full_surv[full_surv["ajcc_pathologic_tumor_stage"].str.contains("Stage")]
    full_surv=full_surv[~(full_surv["ajcc_pathologic_tumor_stage"].isin(["Stage X"]))]
    #split A,B,C from stage designations
    full_surv["ajcc_pathologic_tumor_stage"]=full_surv["ajcc_pathologic_tumor_stage"].str.split("A").str[0]
    full_surv["ajcc_pathologic_tumor_stage"]=full_surv["ajcc_pathologic_tumor_stage"].str.split("B").str[0]
    full_surv["ajcc_pathologic_tumor_stage"]=full_surv["ajcc_pathologic_tumor_stage"].str.split("C").str[0]
    print(full_surv.shape)
    full_surv=full_surv.join(pd.get_dummies(full_surv["ajcc_pathologic_tumor_stage"]))
    #remap gender
    full_surv["gender"]=full_surv["gender"].map({"MALE":0,"FEMALE":1})
    
    full_surv=full_surv[~full_surv[surv_type].isnull()]
    full_surv=full_surv[~full_surv[surv_type+".time"].isnull()]
    
    full_surv=full_surv[~full_surv["age_at_initial_pathologic_diagnosis"].isnull()]
    
    #map to cancer type
    canc = pd.read_csv("/cellar/users/mpagadal/gtype.meta", delimiter="\t", header=None)
    mp = canc.set_index(2)[0].to_dict()
    full_surv["cancer"]=full_surv["FID"].map(mp)
    full_surv=full_surv[~full_surv["cancer"].isnull()]
    
    full_surv['ajcc_pathologic_tumor_stage'] = full_surv['ajcc_pathologic_tumor_stage'].replace({'Stage 0':0,'Stage I': 1,'Stage II': 2,'Stage III': 3,'Stage IV': 4})
    
    return(full_surv)

In [7]:
def run_surv_plot(df_annot,surv_df,surv_type,out):
    
    '''
    plot kaplan meier for each snp in df_annot
    '''
    for i,row in df_annot.iterrows():
        try:
            surv_df_canc=surv_df[surv_df["cancer"]==row["cancer"]]
            x=[x for x in surv_df.columns if row["snps"].rsplit(":",2)[0] in x][0]
            rs=x.rsplit(":",2)[0]
        
            allele1=x.split(":")[2]
            allele2=str(x.split("_")[0]).split(":")[3]
            minor=x.split("_")[1]
            if allele1 == minor:
                major=allele2
            else:
                major=allele1
            
            fig=plt.figure(figsize=(12,5))
            ax1 = plt.subplot(1,2,1)
            
            groups = surv_df_canc[x]
            ix0 = (groups == 0)
            ix1 = (groups == 1)
            ix2 = (groups == 2)

            kmf = KaplanMeierFitter()
            kmf.fit(surv_df_canc[surv_type+'.time'][ix2], surv_df_canc[surv_type][ix2],label=minor+minor)
            kmf.plot(color='green', ci_show=False, ax=ax1)
            
            kmf.fit(surv_df_canc[surv_type+'.time'][ix1], surv_df_canc[surv_type][ix1],label=major+minor)
            kmf.plot(ax=ax1, color='orange', ci_show=False)
    
            kmf.fit(surv_df_canc[surv_type+'.time'][ix0], surv_df_canc[surv_type][ix0],label=major+major)
            kmf.plot(ax=ax1, color='blue', ci_show=False)

            results = logrank_test(surv_df_canc[surv_type+'.time'][ix0], surv_df_canc[surv_type+'.time'][ix2],event_observed_A=surv_df_canc[surv_type][ix0], event_observed_B=surv_df_canc[surv_type][ix2], alpha=.95) 
            plt.title('{}\n{}\nLog-rank test: p<{:.3}'.format(x,row["cancer"],results.p_value))

            plt.ylabel('% survival')
            plt.xlabel(surv_type+' time (days)')
            plt.legend(frameon=False)

            plt.tight_layout()

            plt.savefig(out+surv_type+"."+x+"."+row["cancer"]+'.pdf')
            plt.close()
        except:
            pass
        

In [8]:
def fdr(df,pseudo,p):
    
    '''
    calculate Benjamini-Hochberg FDR
    pseudo: True or False
    '''
    
    #add pseudocounta
    if pseudo == True:
        df[p]=df[p]+0.000000000001
    
    #calculate benjamin-hochberg fdr
    probabilities = df[p].to_numpy()
    report = multi.multipletests(probabilities,
                alpha=0.05,
                method="fdr_bh",
                is_sorted=False,
            )
    discoveries = report[1]
    df["fdr"]=discoveries
    df["fdr"]=pd.to_numeric(df["fdr"])
    df["-log10(fdr)"]=-np.log10(df["fdr"])
    
    return(df)

### Get associations

In [9]:
snps=pd.read_csv("../tables/time-germline.csv",delimiter="\t")
snps["snp_noallele"]=snps["proxy_variant"].str.rsplit(":",2).str[0]

In [10]:
fdr_thresh=0.05

In [11]:
surv_fdr=pd.read_csv("../tables/supplementary_table_11.csv",sep=",")

In [12]:
prog_snps=surv_fdr[surv_fdr["logrank p"]<fdr_thresh]["snps"].unique().tolist()
prog_snps=[x.split("_")[0] for x in prog_snps]

In [13]:
#check how many snps are associated with survival in multiple cancer types
surv_number=surv_fdr[surv_fdr["logrank p"]<fdr_thresh][["snps","cancer"]].drop_duplicates().groupby(["snps"]).size().reset_index()
surv_number=surv_number.sort_values(by=0,ascending=False)
surv_number["snp_noallele"]=surv_number["snps"].str.rsplit(":",2).str[0]
snp_sig=snps[snps["snp_noallele"].isin(surv_number[surv_number[0]>1]["snp_noallele"].tolist())]

In [14]:
snp_sig=snp_sig.loc[~snp_sig["proxy_variant"].duplicated()]

In [15]:
surv=pd.read_csv("../data/survival/clinical_PANCAN_patient_with_followup.tsv",delimiter="\t",encoding="unicode_escape")
keep=pd.read_csv("../data/survival/surv_var_keep_MP.csv") #keep only curated survival variables
surv_clean=surv[["acronym"]+keep["var"].tolist()]

total_counts=pd.DataFrame()
for x in surv_clean.columns[2:]:
    surv_not_null=surv_clean[~surv_clean[x].isnull()]
    print(x)
    try:
        surv_not_null=surv_not_null[~(surv_not_null[x].str.contains("Not Av"))]
        surv_not_null=surv_not_null[~(surv_not_null[x].str.contains("Not Ap"))]
        surv_not_null=surv_not_null[~(surv_not_null[x].str.contains("Unk"))]
        df_counts=surv_not_null["acronym"].value_counts().reset_index()
        df_counts["var"]=x
        total_counts=total_counts.append(df_counts)
    except:
        df_counts=surv_not_null["acronym"].value_counts().reset_index()
        df_counts["var"]=x
        total_counts=total_counts.append(df_counts)
supp_table=total_counts.groupby('index')['var'].apply(lambda x: ','.join(x)).reset_index()
supp_table.loc[14]["var"]='history_of_neoadjuvant_treatment,radiation_therapy,height,weight,T2_pathologic_T,T3_pathologic_T,T4_pathologic_T,M1_pathologic_M,N1_pathologic_N,G1_neoplasm_histologic_grade,G2_neoplasm_histologic_grade,G3_neoplasm_histologic_grade,G4_neoplasm_histologic_grade,alcohol,NAFLD,hepatitis'
supp_table.loc[15]["var"]='radiation_therapy,targeted_molecular_therapy,T2_pathologic_T,T3_pathologic_T,T4_pathologic_T,M1_pathologic_M,N1_pathologic_N,N2_pathologic_N,N3_pathologic_N,Current smoker_tobacco_smoking_history,Lifelong Non-smoker_tobacco_smoking_history'
supp_table.loc[26]["var"]='radiation_therapy,history_of_undescended_testis,T2_pathologic_T,M1_pathologic_M,N1_pathologic_N,N2_pathologic_N'
supp_table.loc[29]["var"]='radiation_therapy,height,weight,hypertension,G2_neoplasm_histologic_grade,G3_neoplasm_histologic_grade,High Grade_neoplasm_histologic_grade,Peri (6-12 months since last menstrual period)_menopause_status,Post (prior bilateral ovariectomy OR >12 mo since LMP with no prior hysterectomy)_menopause_status,Pre (<6 months since LMP AND no prior bilateral ovariectomy AND not on estrogen replacement)_menopause_status'

for x in surv_clean.columns:
    try:
        m = surv_clean[x].str.contains('\[', na=False)
        surv_clean.loc[m,x] = np.nan
    except:
        pass

# map variables to category
mp_map={'Equivocal':np.nan,
 'Indeterminate':np.nan,
 'NO':0,
 'Negative':0,
 'No':0,
 'Not Performed':np.nan,
 'Positive':1,
 'YES':1,
 'Yes':1,
 'Yes, Pharmaceutical Treatment Prior to Resection':1,
 'Yes, Radiation Prior to Resection':1,
 'Yes; bilateral':1,
 'Yes; left testicle only':1,
 'Yes; right testicle only':1,
 'negative':0,
 'positive':1,
}
for var in keep[keep["coding"]=="map"]["var"].unique():
    surv_clean[var]=surv_clean[var].map(mp_map)  

#clean staging
surv_clean["pathologic_T"]=surv_clean["pathologic_T"].str[0:2]
surv_clean["pathologic_N"]=surv_clean["pathologic_N"].str[0:2]
surv_clean["pathologic_M"]=surv_clean["pathologic_M"].str[0:2]
surv_clean["pathologic_stage"]=surv_clean["pathologic_stage"].str.replace("A","")
surv_clean["pathologic_stage"]=surv_clean["pathologic_stage"].str.replace("B","")
surv_clean["pathologic_stage"]=surv_clean["pathologic_stage"].str.replace("C","")
surv_clean["pathologic_stage"]=surv_clean["pathologic_stage"].replace("Stage X",np.nan)
surv_clean['pathologic_stage'] = surv_clean['pathologic_stage'].replace({'Stage 0':0,'Stage I': 1,'Stage II': 2,'Stage III': 3,'Stage IV': 4,'Stage IS':0,'I or II NOS':0})

#dummy code specified variables
mp_dummy={}
for x in keep[keep["coding"]=="dummy"]["var"].unique():
    print(x)
    df_dummy=pd.get_dummies(surv_clean[x])
    df_dummy.columns=[col+"_"+x for col in df_dummy.columns]
    mp_dummy[x]=df_dummy.columns.tolist()
    surv_clean=surv_clean.join(df_dummy)
    del surv_clean[x]
    if "None" in surv_clean.columns:
        del surv_clean["None"]
    if "Other" in surv_clean.columns:
        del surv_clean["Other"]
    
surv=pd.read_csv("/cellar/users/mpagadal/resources/from-labmembers/andrea/Liu2018.TCGA_survival.csv")
surv=surv[["bcr_patient_barcode","gender","age_at_initial_pathologic_diagnosis","type","OS","OS.time","PFI","PFI.time"]]
surv=pd.merge(surv,surv_clean,on="bcr_patient_barcode",how="left")
surv["gender"]=surv["gender"].map({"MALE":0,"FEMALE":1})

del surv["T0_pathologic_T"]
del surv["TX_pathologic_T"]
del surv["Ti_pathologic_T"]
del surv["T1_pathologic_T"]

del surv["MX_pathologic_M"]
del surv["cM_pathologic_M"]
del surv["M0_pathologic_M"]

del surv["N0_pathologic_N"]
del surv["NX_pathologic_N"]

del surv["Indeterminate (neither Pre or Postmenopausal)_menopause_status"]
del surv["No History of Primary Risk Factors_hist_hepato_carc_fact"]
del surv["Other_hist_hepato_carc_fact"]
del surv["lymph_node_examined_count"]
del surv["synchronous_colon_cancer_present"]
del surv["pathologic_stage"]

del surv["GX_neoplasm_histologic_grade"]
del surv["G1_neoplasm_histologic_grade"]

del surv["Current Reformed Smoker, Duration Not Specified_tobacco_smoking_history"]
del surv["Current reformed smoker for < or = 15 years_tobacco_smoking_history"]
del surv["Current reformed smoker for > 15 years_tobacco_smoking_history"]
del surv["history_of_neoadjuvant_treatment"]

surv["alcohol"]=surv[[x for x in surv.columns if "Alcohol consumption" in x]].sum(axis=1)
surv["NAFLD"]=surv[[x for x in surv.columns if "Nonalcoholic Fatty Liver Diseas" in x]].sum(axis=1)
surv["hepatitis"]=surv[[x for x in surv.columns if "Hepatitis" in x]].sum(axis=1)

/cellar/users/tsears/miniconda3/envs/ML/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,9,27,42,44,45,46,47,48,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273

history_of_neoadjuvant_treatment
radiation_therapy
pathologic_T
pathologic_M
pathologic_N
pathologic_stage
lymph_node_examined_count
number_of_lymphnodes_positive_by_he
neoplasm_histologic_grade
height
weight
tobacco_smoking_history
targeted_molecular_therapy
menopause_status
breast_carcinoma_progesterone_receptor_status
breast_carcinoma_estrogen_receptor_status
lab_proc_her2_neu_immunohistochemistry_receptor_status
lab_procedure_her2_neu_in_situ_hybrid_outcome_type
diagnosis
alcohol_history_documented
synchronous_colon_cancer_present
hypertension
seizure_history
history_ionizing_rt_to_head
headache_history
extrathyroid_carcinoma_present_extension_status
gleason_score
diagnosis_subtype
history_hepato_carcinoma_risk_factor
ct_scan
history_of_undescended_testis
history_hypospadias
history_myasthenia_gravis
mitotane_therapy
ebv_antibody_status
relative_family_cancer_history
hist_hepato_carc_fact
creatinine_value_in_mg_dl


/cellar/users/tsears/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/cellar/users/tsears/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/cellar/users/tsears/miniconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/cellar/users/tsears/mini

pathologic_T
pathologic_M
pathologic_N
neoplasm_histologic_grade
tobacco_smoking_history
menopause_status
extrathyroid_carcinoma_present_extension_status
diagnosis_subtype
history_hepato_carcinoma_risk_factor
hist_hepato_carc_fact


In [16]:
#merge snps onto clinical file
raw=pd.read_csv("../data/genotypes/all.time.raw",delim_whitespace=True)
raw=raw.rename(columns={"FID":"bcr_patient_barcode"})
surv=pd.merge(surv,raw,on="bcr_patient_barcode")

In [17]:
#get snps
prog_snps=surv_fdr[surv_fdr["logrank p"]<fdr_thresh]["snps"].unique().tolist()
prog_snps=[x.split("_")[0] for x in prog_snps]

rsid_lst=[]
for snp in prog_snps:
    try:
        rsid_lst.append([x for x in surv.columns if snp.rsplit(":",2)[0] in x][0])
    except:
        print(snp)

In [18]:
canc_counts=surv_clean["acronym"].value_counts().reset_index()
canc_keep=canc_counts[canc_counts["acronym"]>100]["index"].tolist()

In [19]:
#Create OS surv dataframe and split 70/30 by each cancer type
os_surv=make_surv_df("/cellar/users/mpagadal/resources/from-labmembers/andrea/Liu2018.TCGA_survival.csv","OS","/cellar/users/mpagadal/projects/germline-immune/data/genotypes/all.time.raw")
pfi_surv=make_surv_df("/cellar/users/mpagadal/resources/from-labmembers/andrea/Liu2018.TCGA_survival.csv","PFI","/cellar/users/mpagadal/projects/germline-immune/data/genotypes/all.time.raw")

(6875, 976)
(6875, 976)


In [20]:
np.unique(os_surv["cancer"])

array(['ACC', 'BLCA', 'BRCA', 'CHOL', 'COAD', 'ESCA', 'HNSC', 'KICH',
       'KIRC', 'KIRP', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'PAAD', 'READ',
       'SKCM', 'STAD', 'TGCT', 'THCA', 'UVM'], dtype=object)

### PFI survival analysis

In [21]:
pfi_test=pd.DataFrame()
pfi_train=pd.DataFrame()

for cancer in np.unique(pfi_surv["cancer"]):
    
    temp_train,temp_test=train_test_split(pfi_surv[pfi_surv["cancer"]==cancer],test_size=0.3,random_state=random_state)
    pfi_test=pfi_test.append(temp_test)
    pfi_train=pfi_train.append(temp_train)

surv_filt=surv.drop(["gender","age_at_initial_pathologic_diagnosis","type","OS","OS.time","PFI","PFI.time","acronym"],axis=1)
surv_filt=surv_filt.drop(surv_filt.filter(like=":").columns.values,axis=1)

#merge additional covariates into df
pfi_test=pd.merge(pfi_test,surv_filt,left_on="FID",right_on="bcr_patient_barcode")
pfi_train=pd.merge(pfi_train,surv_filt,left_on="FID",right_on="bcr_patient_barcode")

In [ ]:
#run coxph analysis on test datasets, keep cov column for analysis on training datasets

df_coxph = pd.DataFrame()
cancer_test=[]
cov_cols=[]

for i,row in supp_table[supp_table["index"].isin(canc_keep)].iterrows():
    print(row["index"])
    
    for snp in rsid_lst:
        temp_df = pfi_test[pfi_test["cancer"]==row["index"]]
        temp_df=temp_df[~temp_df[snp].isnull()]
        
        if len(temp_df[snp].value_counts())>1:       
            cov_col=row["var"].split(",")
            for x in cov_col:
                if x in [x for x in mp_dummy.keys()]:
                    cov_col=cov_col+mp_dummy[x]
            cov_col=[x for x in cov_col if x in surv.columns]
            cov_col=[x for x in cov_col if len(temp_df[x].value_counts()) > 1]
            
            if len(temp_df["gender"].value_counts())>1:
                temp_df = temp_df[['age_at_initial_pathologic_diagnosis',snp,"gender","PFI",'PFI.time']+cov_col]
            else:
                temp_df = temp_df[['age_at_initial_pathologic_diagnosis',snp,"PFI",'PFI.time']+cov_col]
                
            #possibly use this line?
            #temp_df=temp_df.drop(list(temp_df.var().index[temp_df.var()<0.01]),axis=1)
            
            temp_df=temp_df.fillna(0)
            cph = CoxPHFitter(penalizer=0.01)
            cph.fit(temp_df, event_col="PFI", duration_col='PFI.time')
            cancer_test.append(row["index"])
            cov_cols.append(cov_col)
            df_coxph  = df_coxph.append(cph.summary.loc[[snp]])


BLCA
BRCA


/cellar/users/tsears/miniconda3/envs/ML/lib/python3.7/site-packages/lifelines/utils/__init__.py:1122: ConvergenceWarning: Column Peri (6-12 months since last menstrual period)_menopause_status have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['PFI'].astype(bool)
>>> print(df.loc[events, 'Peri (6-12 months since last menstrual period)_menopause_status'].var())
>>> print(df.loc[~events, 'Peri (6-12 months since last menstrual period)_menopause_status'].var())

A very low variance means that the column Peri (6-12 months since last menstrual period)_menopause_status completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/cellar/users/tsears/miniconda3/envs/ML/lib/python3.7/site

CESC
COAD
ESCA
GBM
HNSC


In [ ]:
x=pd.DataFrame(zip(cancer_test,cov_cols))
cov_col_table=x[~x[0].duplicated()]
cov_col_table=cov_col_table.set_index(0)
cov_col_table["index"]=cov_col_table.index

In [ ]:
#run coxph analysis on TRAIN datasets, use cov column from analysis on test datasets

df_coxph_train = pd.DataFrame()
cancer_train=[]

for i,row in cov_col_table.iterrows():
    print(row["index"])
    
    for snp in rsid_lst:
        temp_df = pfi_train[pfi_train["cancer"]==row["index"]]
        temp_df=temp_df[~temp_df[snp].isnull()]
        
        cov_col=row[1]
            
        if max(temp_df[snp])==0:
            continue
        if len(temp_df["gender"].value_counts())>1:
            temp_df = temp_df[['age_at_initial_pathologic_diagnosis',snp,"gender","PFI",'PFI.time']+cov_col]
        else:
            temp_df = temp_df[['age_at_initial_pathologic_diagnosis',snp,"PFI",'PFI.time']+cov_col] 
        
        temp_df=temp_df.drop(list(temp_df.var().index[temp_df.var()<0.01]),axis=1) #seems to be necessary

        temp_df=temp_df.fillna(0)
        cph = CoxPHFitter(penalizer=0.01)
        cph.fit(temp_df, event_col="PFI", duration_col='PFI.time')
        cancer_train.append(row["index"])
        df_coxph_train  = df_coxph_train.append(cph.summary.loc[[snp]])


In [ ]:
#Insert snp character column, add cancer index
df_coxph["proxy_variant_chars"]=df_coxph.index
df_coxph_train["proxy_variant_chars"]=df_coxph_train.index

df_coxph["cancer"]=cancer_test
df_coxph_train["cancer"]=cancer_train

In [ ]:
#Apply beta values from respective coxph dataset to respective surv dataset
pfi_test_beta=pfi_test.copy()
pfi_train_beta=pfi_train.copy()

for i,row in df_coxph.iterrows():
    if row["proxy_variant_chars"] in pfi_test and not np.isnan(row["coef"]):
        pfi_test_beta.loc[pfi_test_beta["cancer"]==row["cancer"],i]=pfi_test.loc[pfi_test["cancer"]==row["cancer"],i]*row["coef"]

for i,row in df_coxph_train.iterrows():
    if row["proxy_variant_chars"] in pfi_train and not np.isnan(row["coef"]):
        pfi_train_beta.loc[pfi_train_beta["cancer"]==row["cancer"],i]=pfi_train[i][pfi_train["cancer"]==row["cancer"]]*row["coef"]

In [ ]:
#Loop through lasso, drop snps, input list of hyperparams as well as respective cancer types
#store coxph outcomes into final df for easy final plotting
#run for loop by index and use 3 separate lists for input info?

#### part 1, lasso model ####
cancer_list=cov_col_table["index"]
cancer_list=cancer_list.drop("TGCT")
km_curve="off"


alphas=[0.0001,0.005,0.01,0.000005,0.001,0.0001,0.0001,0.01,0.00001,0.009,0.0001,0.0001,0.00001,0.00001,0.001,0.001]
#.      BLCA.   BRCA. COAD     ESCA.   HNSC  KICH  KIRC.  KIRP.  LIHC  LUAD.   LUSC.  PAAD  READ. SKCM    STAD.  #THCA 


hazard_list=pd.DataFrame()
all_survival_snps=pd.DataFrame()
cohort_sizes_pfs=[]

for i in range(len(cancer_list)):
    
    #Filter down dataframes to snps used in respective cancer type
    surv_fdr_drop=surv_fdr[surv_fdr["cancer"]==cancer_list[i]]
    surv_fdr_drop=surv_fdr_drop[surv_fdr_drop["survival"]=="PFI"]
    surv_fdr_drop=surv_fdr_drop[surv_fdr_drop["logrank p"]<0.05]
    
    cols_to_drop=np.setdiff1d(pfi_train_beta.filter(like=":").columns.values.tolist(),surv_fdr_drop["snps"].to_list()).tolist()
    
    train_df=pfi_train_beta[pfi_train_beta["cancer"]==cancer_list[i]]
    test_df=pfi_test_beta[pfi_test_beta["cancer"]==cancer_list[i]]
    
    train_df=train_df.drop(cols_to_drop,axis=1)
    test_df=test_df.drop(cols_to_drop,axis=1)
    
    print(train_df.shape)
    
    train_y=train_df["PFI"]
    test_y=test_df["PFI"]
    
    train_x=train_df.filter(like=":")
    test_x=test_df.filter(like=":")

    #filter using LASSO regression
    # define mode
    model = Lasso(alpha=alphas[i],random_state=random_state)
    # fit model
    model.fit(train_x, train_y)
    p = model.predict(train_x)  # test the output by changing values

    #Find model that gives best AUC, do a cutoff of weakest contributing SNPS, then run all analysis portions
    preds=p
    fpr, tpr, threshold = metrics.roc_curve(list(train_y), preds,pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)

    print(roc_auc)

    #### part 2 ####

    cohort_sizes_pfs.append([cancer_list[i],test_x.shape[0]])
    #remake SNP list with dropped "useless" SNPs
    
    print("SNPs dropped = ",len(train_x.columns.values[model.coef_==0].tolist()),"out of ",len(train_x.columns.values))
    train_df=train_df.drop(train_x.columns.values[model.coef_==0].tolist(),axis=1)
    test_df=test_df.drop(test_x.columns.values[model.coef_==0].tolist(),axis=1)
    
    surv_fdr_drop=surv_fdr_drop[~surv_fdr_drop["snps"].isin(test_x.columns.values[model.coef_==0].tolist())]
    all_survival_snps=all_survival_snps.append(surv_fdr_drop)
    
    train_x=train_df.filter(like=":")
    test_x=test_df.filter(like=":")
    
    regressor_xgb = xgboost.XGBRFRegressor(n_estimators = 300, random_state = random_state,learning_rate=0.5,max_depth=9)
    regressor_xgb.fit(train_x, train_y)  

    p = regressor_xgb.predict(test_x)  # test the output by changing values

    test_df["LASSO_burden"]=p*100
    cat1 = test_df[test_df["PFI"]==0]
    cat2 = test_df[test_df["PFI"]==1]
    print("mannWhitnetPredScore=",stats.mannwhitneyu(cat1["LASSO_burden"], cat2["LASSO_burden"])[1])

    #now do hazard plot on PRS scores generated...
    test_df_coxPH=test_df[list(np.append(['Stage II', 'Stage III', 'Stage I','ajcc_pathologic_tumor_stage',
            'PFI', 'PFI.time',
           'age_at_initial_pathologic_diagnosis',
           'LASSO_burden'],cov_col_table.loc[cancer_list[i],1]))]
    
    #filter covariates that throw errors due to low variance
    test_df_coxPH=test_df_coxPH.drop(list(test_df_coxPH.var().index[test_df_coxPH.var()<0.01]),axis=1) #seems to be necessary
    test_df_coxPH=test_df_coxPH.fillna(0)

    cph = CoxPHFitter(penalizer=0.01)

    cph.fit(test_df_coxPH, duration_col = 'PFI.time', event_col = 'PFI')
    print(cancer_list[i])
    print(cph.summary.loc["LASSO_burden"])
    
    hazard_list=hazard_list.append(cph.summary.loc["LASSO_burden"])
    
    if km_curve=="on":
        #plot KM curve
        surv_df_canc=test_df_coxPH

        fig=plt.figure(figsize=(5,5))
        ax1 = plt.subplot(1,1,1)

        quantiles=np.quantile(surv_df_canc["LASSO_burden"],[0,0.25,0.75,1])
        surv_df_canc['group'] = pd.cut(surv_df_canc["LASSO_burden"], quantiles, labels=[0,1,2])

        kmf = KaplanMeierFitter()
        kmf.fit(surv_df_canc[surv_df_canc['group']==2]['PFI.time'], surv_df_canc[surv_df_canc['group']==2]['PFI'],label="High, N="+str(surv_df_canc[surv_df_canc['group']==2].shape[0]))
        kmf.plot(ci_show=False, ax=ax1)
        kmf.fit(surv_df_canc[surv_df_canc['group']==1]['PFI.time'], surv_df_canc[surv_df_canc['group']==1]['PFI'],label="Med, N="+str(surv_df_canc[surv_df_canc['group']==1].shape[0]))
        kmf.plot(ci_show=False, ax=ax1)
        kmf.fit(surv_df_canc[surv_df_canc['group']==0]['PFI.time'], surv_df_canc[surv_df_canc['group']==0]['PFI'],label="Low, N="+str(surv_df_canc[surv_df_canc['group']==0].shape[0]))
        kmf.plot(ci_show=False, ax=ax1)

        results = logrank_test(surv_df_canc[surv_df_canc["group"]==2]['PFI.time'], surv_df_canc[surv_df_canc["group"]==0]['PFI.time'],event_observed_A=surv_df_canc[surv_df_canc["group"]==2]['PFI'], event_observed_B=surv_df_canc[surv_df_canc["group"]==0]['PFI'], alpha=.95)
        results2 = logrank_test(surv_df_canc[surv_df_canc["group"]==2]['PFI.time'], surv_df_canc[surv_df_canc["group"]==1]['PFI.time'],event_observed_A=surv_df_canc[surv_df_canc["group"]==2]['PFI'], event_observed_B=surv_df_canc[surv_df_canc["group"]==1]['PFI'], alpha=.95)
        results3 = logrank_test(surv_df_canc[surv_df_canc["group"]==1]['PFI.time'], surv_df_canc[surv_df_canc["group"]==0]['PFI.time'],event_observed_A=surv_df_canc[surv_df_canc["group"]==1]['PFI'], event_observed_B=surv_df_canc[surv_df_canc["group"]==0]['PFI'], alpha=.95)

        plt.ylabel('% survival')
        plt.xlabel('PFI time (days)')
        plt.title(str(cancer_list[i])+"\n"+"High vs Low "+str(np.round(results.p_value,4))+"\n"+"High vs Med "+str(np.round(results2.p_value,4))+"\n"+"Med vs Low "+str(np.round(results3.p_value,4)))
        plt.legend(frameon=False)
        plt.savefig("../plots/fig4/"+cancer_list[i]+"_0.25_0.75_PFI_KM.pdf")
        
#all_survival_snps.to_csv("/cellar/users/tsears/projects/germline-immune/data/fig6/PFS_input_snps.txt",sep="\t")


In [ ]:
haz_plot_=hazard_list
haz_plot_.index=cancer_list
haz_plot_["Cancer Type"]=cancer_list
#haz_plot_=haz_plot_.drop(haz_plot_.index[abs(haz_plot_["coef"])>1][0])
haz_plot_=haz_plot_.drop(["KICH"])
haz_plot_=fdr(haz_plot_,False,"p")
haz_plot_=haz_plot_.sort_values(by="exp(coef)",ascending=True)
plt.figure(figsize=(15,3))
plt.errorbar(x=haz_plot_["Cancer Type"].tolist(),y=haz_plot_["exp(coef)"].tolist(),yerr=haz_plot_["se(coef)"].tolist(),fmt="o")
haz_plot_thresh=haz_plot_[haz_plot_["fdr"]<0.05]
plt.errorbar(x=haz_plot_thresh["Cancer Type"].tolist(),y=haz_plot_thresh["exp(coef)"].tolist(),yerr=haz_plot_thresh["se(coef)"].tolist(),fmt="o",color="red")
plt.xlabel("Cancer Type")
plt.ylabel("Hazard Ratio")
plt.axhline(1,linestyle="--",color="gray")
plt.savefig("../plots/cancer_risk_PFI_hazardV2_long.pdf")

In [ ]:
cohort_sizes_pfs